In [1]:
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm

In [2]:
!ls -l ../data/

total 580596
-rw------- 1 v.bugaevskii v.bugaevskii    904039 Apr 17 11:12 appl.csv
-rw------- 1 v.bugaevskii v.bugaevskii   6577695 Apr 17 11:12 aum.csv
-rw------- 1 v.bugaevskii v.bugaevskii  91636746 Apr 17 11:12 balance.csv
-rw------- 1 v.bugaevskii v.bugaevskii   1110765 Apr 17 11:12 client.csv
-rw------- 1 v.bugaevskii v.bugaevskii   6686165 Apr 17 11:12 com.csv
-rw------- 1 v.bugaevskii v.bugaevskii   7499804 Apr 17 11:12 deals.csv
-rw------- 1 v.bugaevskii v.bugaevskii     18979 Apr 17 11:12 dict_mcc.csv
-rw------- 1 v.bugaevskii v.bugaevskii   2322389 Apr 17 11:12 funnel.csv
-rw------- 1 v.bugaevskii v.bugaevskii  10553763 Apr 17 11:12 payments.csv
-rw------- 1 v.bugaevskii v.bugaevskii 467193115 Apr 17 11:12 trxn.csv


In [3]:
from pathlib import Path

In [4]:
path_df = Path('../data').rglob('*.csv')
path_df = sorted(path_df)

for path in tqdm(path_df):
    df_name = 'df_' + path.name.rsplit('.', 1)[0]
    vars()[df_name] = pd.read_csv(path)
    print(f'{path} saved to {df_name}; {df_name}.shape = {vars()[df_name].shape}')

  0%|          | 0/10 [00:00<?, ?it/s]

../data/appl.csv saved to df_appl; df_appl.shape = (12030, 6)
../data/aum.csv saved to df_aum; df_aum.shape = (117392, 4)
../data/balance.csv saved to df_balance; df_balance.shape = (1194684, 9)
../data/client.csv saved to df_client; df_client.shape = (21498, 8)
../data/com.csv saved to df_com; df_com.shape = (113055, 10)
../data/deals.csv saved to df_deals; df_deals.shape = (109016, 8)
../data/dict_mcc.csv saved to df_dict_mcc; df_dict_mcc.shape = (915, 3)
../data/funnel.csv saved to df_funnel; df_funnel.shape = (21498, 16)
../data/payments.csv saved to df_payments; df_payments.shape = (188068, 4)


/home/v.bugaevskii/.conda/envs/idao2021/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


../data/trxn.csv saved to df_trxn; df_trxn.shape = (3035705, 11)


In [5]:
df_grouped = df_payments.groupby(['client_id', 'pmnts_name']).agg(['mean', 'count'])
df_grouped.columns = ['_'.join(col) for col in df_grouped.columns]
df_grouped = df_grouped.reset_index()
df_grouped.head()

,client_id,pmnts_name,sum_rur_mean,sum_rur_count
0,-9220236243053692422,Salary receipts,15737.333333,9
1,-9220233431709087652,Salary receipts,32787.000000,23
2,-9219699286371310531,Salary receipts,4629.636364,11
3,-9218871523310554579,Salary receipts,15783.120000,25
4,-9217476774110203314,Salary receipts,3614.833333,6


In [6]:
df_features_pull = []

for values in ['sum_rur_mean', 'sum_rur_count']:
    df_features_ = pd.pivot_table(
        df_grouped,
        values=values,
        index='client_id', 
        columns='pmnts_name',
    )
    df_features_.columns = df_features_.columns.str.replace(' ', '_')
    df_features_ = df_features_.add_prefix('pmnts_name_' + values + '_')
    df_features_pull.append(df_features_)

df_features = pd.concat(df_features_pull, axis=1)
df_features.head()

pmnts_name,pmnts_name_sum_rur_mean_Pension_receipts,pmnts_name_sum_rur_mean_Salary_receipts,pmnts_name_sum_rur_count_Pension_receipts,pmnts_name_sum_rur_count_Salary_receipts
client_id,,,,
-9220236243053692422,NaN,15737.333333,NaN,9.0
-9220233431709087652,NaN,32787.000000,NaN,23.0
-9219699286371310531,NaN,4629.636364,NaN,11.0
-9218871523310554579,NaN,15783.120000,NaN,25.0
-9217476774110203314,NaN,3614.833333,NaN,6.0


In [7]:
df_features = df_features.fillna(0)

df_features['pmnts_name_sum_rur_count_Pension_receipts'] = \
    df_features['pmnts_name_sum_rur_count_Pension_receipts'].astype(int)

df_features['pmnts_name_sum_rur_count_Salary_receipts'] = \
    df_features['pmnts_name_sum_rur_count_Salary_receipts'].astype(int)

df_features.head()

pmnts_name,pmnts_name_sum_rur_mean_Pension_receipts,pmnts_name_sum_rur_mean_Salary_receipts,pmnts_name_sum_rur_count_Pension_receipts,pmnts_name_sum_rur_count_Salary_receipts
client_id,,,,
-9220236243053692422,0.0,15737.333333,0,9
-9220233431709087652,0.0,32787.000000,0,23
-9219699286371310531,0.0,4629.636364,0,11
-9218871523310554579,0.0,15783.120000,0,25
-9217476774110203314,0.0,3614.833333,0,6


In [8]:
df_features.to_csv('payments_features.csv', index=True)